# Lesson 2: Storing and Managing Text Chunks in Vector Databases

## Loading and chunking text

## Building a ChromaDB Collection

## Dynamic Chunk Management in ChromaDB

## Dynamic Document Management in ChromaDB